<a href="https://colab.research.google.com/github/nithecs-biomath/mini-schools/blob/main/cube_prac_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUILDING DATA CUBES
## NITheCS mini school: lecture 2

### Install missing packages

In [ ]:
%pip install pygbif

### Only execute the following block when using the TPU kernel

In [ ]:
%pip install geopandas
%pip install pydrive
%pip install ee
%pip install eerepr
%pip install geemap

### Loading packages

In [ ]:
from pygbif import occurrences as occ
import pandas as pd
import geopandas as gpd
from pyproj import Proj, Transformer
from shapely.geometry import mapping
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import io
import zipfile
import math

### Loading Earth Engine

In [ ]:
import ee
import eerepr
import geemap

ee.Authenticate(force=True)
ee.Initialize(project='nithecs-436810')

LANDSAT_ID = "LANDSAT/LC08/C02/T1_L2"
BOUNDARIES_ID = 'FAO/GAUL/2015/level1'
WDPA_ID = 'WCMC/WDPA/current/polygons'
SENTINEL_ID = "COPERNICUS/S2_SR_HARMONIZED"


dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2021-05-01', '2021-06-01')
sa = ee.FeatureCollection(BOUNDARIES_ID).filter(
    'ADM0_NAME == "South Africa"')

dataset_eo = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2020-01-01', '2020-01-30')

protected_areas = ee.FeatureCollection(WDPA_ID)


sa_landsat = dataset.filterBounds(sa)
sa_sentinel = dataset_eo.filterBounds(sa)


### Example of the GBIF API through pygbif

In [ ]:
from pygbif import occurrences
data = occurrences.search(speciesKey=5229490, limit=10)

print(data['results'])

## GBIF data Cubes

### Generating the Cube

#### Exemplar JSON query for generating a data *cube*



```
{
  "sendNotification": true,
  "notificationAddresses": [
    "maarten.trekels@plantentuinmeise.be"
  ],
  "format": "SQL_TSV_ZIP",
  "sql": "SELECT  PRINTF('%04d-%02d', \"year\", \"month\") AS yearMonth,
   GBIF_EEARGCode(10000, decimalLatitude,  decimalLongitude,  COALESCE(coordinateUncertaintyInMeters, 1000) ) AS eeaCellCode,
   speciesKey,
   species,
   establishmentMeans,
   degreeOfEstablishment,
   pathway,
   COUNT(*) AS occurrences,
   COUNT(DISTINCT recordedBy) AS distinctObservers
   FROM  occurrence
   WHERE occurrenceStatus = 'PRESENT'
   AND countryCode = 'BE'
   AND hasCoordinate = TRUE
   AND NOT ARRAY_CONTAINS(issue, 'ZERO_COORDINATE')
   AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_OUT_OF_RANGE')
   AND NOT ARRAY_CONTAINS(issue, 'COORDINATE_INVALID')
   AND NOT ARRAY_CONTAINS(issue, 'COUNTRY_COORDINATE_MISMATCH')
   AND \"month\" IS NOT NULL
   GROUP BY yearMonth,
   eeaCellCode,
   speciesKey,
   species,
   establishmentMeans,
   degreeOfEstablishment,
   pathway
   ORDER BY  yearMonth DESC,
   eeaCellCode ASC,
   speciesKey ASC"
}
```



#### Using the GBIF Download UI

https://techdocs.gbif.org/en/data-use/api-sql-downloads

## Loading the Data cube in pandas



#### Download from GitHub

You can download a pre generated data cube from GitHub or any other online resource

In [ ]:
#data = pd.read_csv('https://raw.githubusercontent.com/nithecs-biomath/mini-schools/refs/heads/main/data/sample_data_SA.csv', sep='\t')

#print(data)

#### Download from Google Drive

In [ ]:
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/Shareddrives/NiTheCS mini school/demo_data/Cube_ZA_QDGC_l3.csv', sep='\t')


In [ ]:
print(data)

## Getting a Geopackage file from the Grid that you use

In [ ]:
# Load QDGC code

input_file = "/content/drive/Shareddrives/NiTheCS mini school/demo_data/qdgc_south_africa.gpkg"

qdgc_ref = gpd.read_file(input_file, layer='tbl_qdgc_03')

In [ ]:
print(qdgc_ref)

## Merging the Data cube with the grid

In [ ]:
#testing if I can merge data and qdgc

test_merge = pd.merge(data, qdgc_ref, left_on='qdgccode', right_on='qdgc')

print(test_merge)


In [ ]:
# Convert to GeoDataFrame

gdf = gpd.GeoDataFrame(test_merge, geometry='geometry')


In [ ]:
# set the CRS and convert to a GeoParquet file
gdf = gdf.set_crs(epsg=4326, inplace=False)

gdf.to_parquet('/content/drive/Shareddrives/NiTheCS mini school/demo_data/data_ZA.parquet')

## Loading data directly as Parquet files


In [ ]:
drive.mount('/content/drive')

gbif_cube = '/content/drive/Shareddrives/NiTheCS mini school/demo_data/data_ZA.parquet'
gbif_points = '/content/drive/Shareddrives/NiTheCS mini school/demo_data/data_ZA_occurrence.parquet'

gdf_cube = gpd.read_parquet(gbif_cube)
gdf_point = gpd.read_parquet(gbif_points)

## Filtering data (e.g. on species)

Check for a single species (Acacia implexa Benth.: https://www.gbif.org/species/2979232)



In [ ]:
#check for a single species
gdf_cube = gdf_cube[gdf_cube['specieskey'].eq(2979232)]
gdf_point = gdf_point[gdf_point['speciesKey'].eq(2979232.0)]


## Apply the function to create a list of features

In [ ]:
# Convert GeoDataFrames to GeoJSON FeatureCollections
def gdf_to_ee_featurecollection(gdf):
    return geemap.geopandas_to_ee(gdf, geodesic=False)

# Assuming gdf1 and gdf2 are your GeoDataFrames
fc1 = gdf_to_ee_featurecollection(gdf_cube)
fc2 = gdf_to_ee_featurecollection(gdf_point)

## Visualization of the data cubes on a map with different layers

In [ ]:
# Create a map centered on your data
center = (-33.97, 18.58)

Map = geemap.Map(layout={"height": "500px", "width": "1000px"}, center=center, zoom=10)


visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Add FeatureCollections to the map
Map.addLayer(fc1, {'color': 'red'}, 'Data Cube')
Map.addLayer(fc2, {'color': 'blue'}, 'Point data')


#Map.addLayer(sa_sentinel, visualization, 'RGB Sentinel 2')

#Map.addLayer(protected_areas)


# Visualise the map
Map

In [ ]:
### Test with NetCDF format

# EBV data cubes in NetCDF format

https://portal.geobon.org/datasets

In [ ]:
%pip install netCDF4

In [ ]:
%pip install rioxarray
%pip install cartopy
%pip install basemap

In [ ]:
import netCDF4 as nc
import xarray as xr


birds_file = xr.open_dataset('/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc')

print(birds_file)

In [ ]:
def print_netcdf_structure(nc_file_path):
  """Prints the structure (groups, variables, and their paths) of a NetCDF file.

  Args:
    nc_file_path: Path to the NetCDF file.
  """
  def print_group_structure(group, path=""):
    """Recursively prints the structure of a group within the NetCDF file."""
    for var_name in group.variables:
      print(f"{path}/{var_name}")  # Print variable path
    for group_name in group.groups:
      subgroup = group.groups[group_name]
      print_group_structure(subgroup, f"{path}/{group_name}")  # Recursively explore subgroups

  with nc.Dataset(nc_file_path, 'r') as nc_file:
    print_group_structure(nc_file)  # Start with the root group

# Example usage:
nc_file_path = '/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc'
print_netcdf_structure(nc_file_path)

In [ ]:
print(birds_file.variables)

In [ ]:
time = birds_file.variables['time']
print(time)

print(birds_file['entity'])

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
from pyproj import Transformer
from matplotlib.colors import ListedColormap, BoundaryNorm

# --- Load Dataset Efficiently ---
birds_file = xr.open_dataset(
    '/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc',
    group="metric_1",
    chunks={'entity': 1, 'time': 1}  # Load only one entity/time slice at a time
)

# --- Select Target Species and Time ---
species_index = 150  # Change to the species index you need
time_index = 0  # Change to the desired time index

# Extract only the required slice
species_data_subset = birds_file['ebv_cube'].sel(entity=species_index, time=time_index).compute()

# Convert to a 2D array
species_distribution_2d = np.squeeze(species_data_subset)

# --- Load Longitude and Latitude (Only Once) ---
with xr.open_dataset('/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc') as ds:
    lon = ds['lon'].values  # 1D array (size: 559)
    lat = ds['lat'].values  # 1D array (size: 437)

# --- Create Meshgrid Efficiently ---
lon_grid, lat_grid = np.meshgrid(lon, lat)

# --- Efficient Coordinate Transformation ---
transformer = Transformer.from_crs("epsg:3035", "epsg:4326", always_xy=True)

# Transform the entire 2D meshgrid
lon_deg, lat_deg = transformer.transform(lon_grid, lat_grid)

# --- Create the Plot ---
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add map features
ax.coastlines()
ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='lightgray')

# --- 🔹 Fix: Use Discrete Colormap Without Color Bar ---
unique_values = np.unique(species_distribution_2d)

# If only one value, choose a single solid color
if len(unique_values) == 1:
    cmap = ListedColormap(["red"])  # Single-color for uniform data
    norm = None
else:
    cmap = ListedColormap(["white", "blue"])  # Adjust colors as needed
    norm = BoundaryNorm([0, 0.5, 1], cmap.N)

# --- 🔹 Fix: Use `shading="nearest"` to Ensure Correct Grid Alignment ---
cs = ax.pcolormesh(
    lon_deg, lat_deg, species_distribution_2d,
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    norm=norm,
    shading='nearest'  # Prevents visual distortion
)

# --- 🔹 Completely Remove Color Bar ---
# No `fig.colorbar(cs)`, so no scale bar will be shown

# --- Get the Species Name Efficiently ---
with xr.open_dataset('/content/drive/Shareddrives/NiTheCS mini school/demo_data/viti_spepop_id77_20240206_v1.nc') as ds:
    species_name = ds['entity'].values[species_index].decode('utf-8').strip()

# --- Final Plot Customization ---
ax.set_title(f"Species {species_name} Distribution (10x10 km grid) at time {birds_file['time'].values[time_index].item()}")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# --- 🔹 Fix: Add Grid Lines to Show Exact 10x10 km Cells ---
gridlines = ax.gridlines(draw_labels=True, linestyle="--", linewidth=0.5, color="black", alpha=0.5)

# Show the plot
plt.show()